In [5]:
import sys
import pandas as pd
import numpy as np
sys.path.append('../script/')
sys.path.append('../class/')
import sunnytools as st
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import siamchartdatareader as scdr
dr=scdr.SiamChartDataReader()
dr.loginToSiamChart()

In [7]:
all_thai_tickers = st.getAllThaiTickers()

In [8]:
len(all_thai_tickers)

650

In [55]:
ticker_list=all_thai_tickers
df_div_all=pd.DataFrame()
df_fs_all=pd.DataFrame()
bad_ticker=[]

for ticker in ticker_list:
    try:
        tryRelogin=False
        while True:
            print('Loading data for ',ticker)
            df_fs=dr.getFinancialData(ticker)
            df_div=dr.getDividendData(ticker)
            print('Done for ticker ',ticker,' size of dividend and fs df are ',len(df_div),len(df_fs))
            if len(df_fs) <= 5 and not tryRelogin:
                try:
                    dr.loginToSiamChart()
                except:
                    print('log in not successful, probably stil in login session')
                tryRelogin=True
                time.sleep(1)
                print("relogin and reload the data")
            else:
                break

        #add ticker
        df_div.insert(0,'ticker',ticker)
        df_fs.insert(0,'ticker',ticker)

        #append to the big dataframe
        df_div_all=df_div_all.append(df_div)
        df_fs_all=df_fs_all.append(df_fs)
    
    except:
        raise
        bad_ticker.append(ticker)

#reorganize columns
prefix_cols=['ticker','revenue','net_profit','eps','dps','gross_profit','ebit','ebitda']
cols=prefix_cols+[x for x in df_fs_all.columns if x not in prefix_cols]
df_fs_all=df_fs_all[cols]

Loading data for  MC
Done for ticker  MC  size of dividend and fs df are  10 17
Loading data for  AOT
Done for ticker  AOT  size of dividend and fs df are  15 54


## SAVE NEW VERSION

In [58]:
## SAVE TO NEW DATA FRAME

## UPDATE/DEBUG DATA

In [90]:
## UPDATE TO OLD DATA FRAME
df_new=df_fs_all
df_old=pd.read_csv('../data/fs/20170701_thai_fs.csv',parse_dates=[0],index_col=[0])
df_old.index=df_old.index.date

new_tickers=df_fs_all.ticker.unique()
print('update information for ticker(s): ',new_tickers)

df_update=df_new.append(df_old[~df_old.ticker.isin(new_tickers)])

#rearrange columns
prefix_cols=['ticker','revenue','net_profit','eps','dps','gross_profit','ebit','ebitda']
cols=prefix_cols+[x for x in df_update.columns if x not in prefix_cols]
df_update=df_update[cols]

update information for ticker(s):  ['MC' 'AOT']


In [91]:
df_update.index[0]

datetime.date(2013, 3, 31)

In [92]:
#double check
df_update.tail()

ticker  revenue  net_profit       eps       dps  gross_profit  \
2016-12-31  UOBKH   326.50       69.55  0.138926 -0.000316           NaN   
2017-03-31  UOBKH   311.71       70.74  0.140000  0.100000           NaN   
2016-09-30  COMAN      NaN         NaN       NaN       NaN           NaN   
2016-12-31  COMAN    44.85       15.17  0.204570  0.000000         34.18   
2017-03-31  COMAN    33.31        5.62  0.040000  0.000000         18.52   

             ebit  ebitda  CurrentRatio   D/E  ...   number_of_shares  \
2016-12-31  86.07   95.93           NaN  0.97  ...         500.625000   
2017-03-31  88.33   98.42           NaN  0.94  ...         505.285714   
2016-09-30    NaN     NaN          1.49  0.37  ...          58.709677   
2016-12-31  15.43   18.50          9.13  0.10  ...          74.155556   
2017-03-31   5.82    8.91          9.36  0.10  ...         140.500000   

            operating_profit  paid_in_capital  property_plant_equipment  \
2016-12-31               NaN           502.45                    122.34   
2017-03-31               NaN           502.45                    116.56   
2016-09-30               NaN            50.00                      3.26   
2016-12-31             15.68            67.00                      4.55   
2017-03-31              4.69            67.00                      4.90   

            return_old  short_term_loan  total_asset  total_equity  \
2016-12-31      240.30              NaN      6220.46       3165.61   
2017-03-31       70.74              NaN      6276.49       3236.35   
2016-09-30       18.20              NaN       190.09        138.25   
2016-12-31       33.37              NaN       448.87        408.19   
2017-03-31        5.62              NaN       451.72        411.05   

            total_liability  year  
2016-12-31          3054.85  2016  
2017-03-31          3040.14  2017  
2016-09-30            51.84  2016  
2016-12-31            40.68  2016  
2017-03-31            40.66  2017  

[5 rows x 51 columns]

In [93]:
#save
from datetime import datetime
outputDir = '../data/fs/'+datetime.today().strftime(format='%Y%m%d')+'_'+'thai_fs.csv'
st.ensure_dir(outputDir)
df_update.to_csv(outputDir)